# Reading data

In [1]:
#!pip install nltk
#!pip install tqdm
#!pip install gensim

In [2]:
import pandas as pd
from functools import reduce
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm_notebook
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import wordnet
from itertools import chain # to flatten list of sentences of tokens into list of tokens
from gensim.models import Phrases
from gensim import corpora
from gensim import models
import numpy as np
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\frank\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
raw_data = pd.read_json('E:\Bigdata\Video_Games_5.json',lines=True)
#test = data.head(200)
#data = data.dropna().reset_index(drop=True)
#raw_data = test

# Cleaning

 Put reviewText and summary together and group by 'asin'

In [4]:
raw_data['review_con'] =  raw_data.summary + ". "+ raw_data.reviewText
data = raw_data[['asin','review_con']]
def splice(series):
       return reduce(lambda x, y: x +". "+ y, series)
data = data.groupby('asin')['review_con'].apply(splice).reset_index()

dict_idx = { asin : idx for idx,asin in enumerate(data['asin']) }

data
dict_idx


{'0700099867': 0,
 '6050036071': 1,
 '7100027950': 2,
 '7293000936': 3,
 '8176503290': 4,
 '907843905X': 5,
 '9625990674': 6,
 '9861019731': 7,
 '9882155456': 8,
 'B000003SQQ': 9,
 'B000006OVE': 10,
 'B000006OVI': 11,
 'B000006OVJ': 12,
 'B000006OVK': 13,
 'B000006OWR': 14,
 'B000006OWS': 15,
 'B000006P0J': 16,
 'B000006P0K': 17,
 'B000006P0M': 18,
 'B000006RGO': 19,
 'B000006RGQ': 20,
 'B000006RGR': 21,
 'B000006RGS': 22,
 'B000007NJC': 23,
 'B000009QD1': 24,
 'B00000DMA8': 25,
 'B00000DMAA': 26,
 'B00000DMAD': 27,
 'B00000DMAE': 28,
 'B00000DMAM': 29,
 'B00000DMAN': 30,
 'B00000DMAO': 31,
 'B00000DMAP': 32,
 'B00000DMAQ': 33,
 'B00000DMAR': 34,
 'B00000DMAT': 35,
 'B00000DMAU': 36,
 'B00000DMAV': 37,
 'B00000DMAX': 38,
 'B00000DMB2': 39,
 'B00000DMB3': 40,
 'B00000DMB4': 41,
 'B00000DMB5': 42,
 'B00000DMB6': 43,
 'B00000DMB9': 44,
 'B00000F1GI': 45,
 'B00000F1GM': 46,
 'B00000G16I': 47,
 'B00000I1B7': 48,
 'B00000I1B8': 49,
 'B00000I1BA': 50,
 'B00000I1BB': 51,
 'B00000I1BE': 52,
 'B

In [5]:
dict_idx['0700099867']

0

In [6]:
data

,asin,review_con
0,0700099867,Pay to unlock content? I don't think so.. Inst...
1,6050036071,"Works good!. Works good, however is not ""like ..."
2,7100027950,Epic Zelda title!. Great game! I love the stor...
3,7293000936,"I want to give it four stars, but.... While th..."
4,8176503290,Not as good as the first one. I enjoyed the fi...
5,907843905X,Full of bugs/no patch to fix them/Vstep forum ...
6,9625990674,Best silicone grip ever!. I still have my sili...
7,9861019731,Perfect!. It works perfectly! Nothing is wrong...
8,9882155456,one. well atleast one of them worked what else...
9,B000003SQQ,Horrid. I bought this game way back and recent...


#### Tokenization

In [7]:
tqdm_notebook().pandas()
data['sentences'] = data.review_con.progress_map(sent_tokenize)
data['sentences'].head(1).tolist()[0][:3] # Print the first 3 sentences of the 1st article

c:\users\frank\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


['Pay to unlock content?',
 "I don't think so..",
 'Installing the game was a struggle (because of games for windows live bugs).Some championship races and cars can only be "unlocked" by buying them as an addon to the game.']

In [8]:
data['tokens_sentences'] = data['sentences'].progress_map(lambda sentences: [word_tokenize(sentence) for sentence in sentences])
print(data['tokens_sentences'].head(1).tolist()[0][:3])


[['Pay', 'to', 'unlock', 'content', '?'], ['I', 'do', "n't", 'think', 'so..'], ['Installing', 'the', 'game', 'was', 'a', 'struggle', '(', 'because', 'of', 'games', 'for', 'windows', 'live', 'bugs', ')', '.Some', 'championship', 'races', 'and', 'cars', 'can', 'only', 'be', '``', 'unlocked', "''", 'by', 'buying', 'them', 'as', 'an', 'addon', 'to', 'the', 'game', '.']]


#### Lemmatizing with POS tagging

In [9]:
data['POS_tokens'] = data['tokens_sentences'].progress_map(lambda tokens_sentences: [pos_tag(tokens) for tokens in tokens_sentences])
print(data['POS_tokens'].head(1).tolist()[0][:3])


[[('Pay', 'NN'), ('to', 'TO'), ('unlock', 'VB'), ('content', 'NN'), ('?', '.')], [('I', 'PRP'), ('do', 'VBP'), ("n't", 'RB'), ('think', 'VB'), ('so..', 'VB')], [('Installing', 'VBG'), ('the', 'DT'), ('game', 'NN'), ('was', 'VBD'), ('a', 'DT'), ('struggle', 'NN'), ('(', '('), ('because', 'IN'), ('of', 'IN'), ('games', 'NNS'), ('for', 'IN'), ('windows', 'NNS'), ('live', 'VBP'), ('bugs', 'NNS'), (')', ')'), ('.Some', 'VBP'), ('championship', 'NN'), ('races', 'NNS'), ('and', 'CC'), ('cars', 'NNS'), ('can', 'MD'), ('only', 'RB'), ('be', 'VB'), ('``', '``'), ('unlocked', 'JJ'), ("''", "''"), ('by', 'IN'), ('buying', 'VBG'), ('them', 'PRP'), ('as', 'IN'), ('an', 'DT'), ('addon', 'NN'), ('to', 'TO'), ('the', 'DT'), ('game', 'NN'), ('.', '.')]]


In [10]:
# Inspired from https://stackoverflow.com/a/15590384


def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [11]:
# Lemmatizing each word with its POS tag, in each sentence
data['tokens_sentences_lemmatized'] = data['POS_tokens'].progress_map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

In [12]:
data['tokens_sentences_lemmatized'].head(1).tolist()[0][:3]

[['Pay', 'to', 'unlock', 'content', '?'],
 ['I', 'do', "n't", 'think', 'so..'],
 ['Installing',
  'the',
  'game',
  'be',
  'a',
  'struggle',
  '(',
  'because',
  'of',
  'game',
  'for',
  'window',
  'live',
  'bug',
  ')',
  '.Some',
  'championship',
  'race',
  'and',
  'car',
  'can',
  'only',
  'be',
  '``',
  'unlocked',
  "''",
  'by',
  'buy',
  'them',
  'as',
  'an',
  'addon',
  'to',
  'the',
  'game',
  '.']]

#### Regrouping tokens and removing stop words

In [13]:
stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 'take', 'use', 'would', 'can']
stopwords_other = ['one', 'mr', 'bbc', 'image', 'getty', 'de', 'en', 'caption', 'also', 'copyright', 'something']
my_stopwords = stopwords.words('English') + stopwords_verbs + stopwords_other

In [14]:
data['tokens'] = data['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
data['tokens'] = data['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>1])

In [15]:
data['tokens'].head(1).tolist()[0][:30]

['pay',
 'unlock',
 'content',
 'think',
 'installing',
 'game',
 'struggle',
 'game',
 'window',
 'live',
 'bug',
 'championship',
 'race',
 'car',
 'unlocked',
 'buy',
 'addon',
 'game',
 'pay',
 'nearly',
 'dollar',
 'game',
 'new',
 'idea',
 'keep',
 'pay',
 'keep',
 'noticed',
 'improvement',
 'physic']

# LDA

## Data preparation

#### Prepare bi-grams and tri-grams

In [16]:
tokens = data['tokens'].tolist()
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])

#### Prepare objects for LDA gensim implementation

In [17]:
dictionary_LDA = corpora.Dictionary(tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]

## Running LDA

In [18]:
np.random.seed(123456)
num_topics = 100
num_words = 20
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

Wall time: 10min 6s


## Quick exploration of LDA results

#### Looking at topics

In [19]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=num_words):
    print(str(i)+": "+ topic)
    print()

0: 0.082*"zombie" + 0.046*"resident_evil" + 0.017*"weapon" + 0.011*"touch_screen" + 0.011*"scary" + 0.011*"horror" + 0.011*"capcom" + 0.010*"survival_horror" + 0.009*"zombies" + 0.008*"dead_rising" + 0.008*"item" + 0.007*"survivor" + 0.007*"mode" + 0.006*"action" + 0.006*"pikmin" + 0.006*"save" + 0.005*"kill" + 0.005*"ammo" + 0.005*"monster" + 0.005*"kill_zombie"

1: 0.148*"steam" + 0.077*"pc" + 0.062*"episode" + 0.038*"download" + 0.028*"disgaea" + 0.026*"install" + 0.019*"portal" + 0.016*"valve" + 0.015*"mod" + 0.015*"computer" + 0.014*"season" + 0.014*"pack" + 0.013*"update" + 0.013*"pc_version" + 0.012*"sale" + 0.011*"windows" + 0.011*"instal" + 0.010*"short" + 0.009*"half_life" + 0.009*"deal"

2: 0.027*"quest" + 0.021*"mmo" + 0.015*"wow" + 0.013*"class" + 0.011*"world" + 0.010*"content" + 0.009*"skill" + 0.009*"pvp" + 0.008*"group" + 0.008*"expansion" + 0.007*"area" + 0.007*"month" + 0.007*"item" + 0.006*"gear" + 0.006*"server" + 0.006*"zone" + 0.006*"free" + 0.006*"craft" + 0.006

#### Allocating topics to documents

In [20]:
print(data.review_con.loc[2666][:])

NES SP RULES!!!!!!!!!!!!!!!!!!!!!!!!!!!. I was thinkin about getting an onyx until i saw it an sp that looked like that musty old(but still awsome ) nes i baught this i dont regret it  a bit so if you are buying an sp get the nes version!!!!. It's great.. It has a nostalgic feel. It looks exactly like a Nintendo Entertainment System,a NES.It's one of my four favorite SP designs,the other being the three Pokemon SP's.The Gameboy Advance SP can play Gameboy,Gameboy Color,and Gameboy Advance games. It also has a backlight.This is a must have for Nintendo gamers,old and new.. The Ultimate Gameboy Experience. If you want to play Mario, Zelda, Kirby, Pok&eacute;mon, Metroid, DonkeyKong, Castlevania, Megaman, and even Sonic the Hedgehog you can either get a Nintendo 3ds( Which is also amazing ) or this. If you had to get any Gameboy get this. Unlike the Original GBA or Gameboy it has a backlight so you can play it at night ( Or indoors ). Not only can it take GBA games it could take Gameboy a

In [21]:
lda_model[corpus[2666]]

[(9, 0.1783507),
 (31, 0.32434937),
 (32, 0.018100403),
 (33, 0.08932274),
 (45, 0.06548626),
 (48, 0.023519719),
 (53, 0.03573968),
 (54, 0.07827279),
 (55, 0.026985904),
 (62, 0.06630751),
 (65, 0.04907757)]

getting topic for a specific asin:

In [22]:
query = '0700099867'
def get_topic(query):
    max_topic = 0
    topic_num = 0
    data_slice = lda_model[corpus[dict_idx[query]]]
    for i, topic in enumerate(data_slice):
        if (data_slice[i][1] > max_topic):
            max_topic = data_slice[i][1]
            topic_num = data_slice[i][0]
    #print (topic_num)
    result = lda_model.print_topic(topic_num)
    #print(result +" match;"+str(max_topic))
    p1 = re.compile(r'["](.*?)["]', re.S)
    result = re.findall(p1, result)
    return result

#get_topic(query)

#### Predicting topics on unseen documents

In [23]:
document = '''fun but this game almost made me snap the disc in half. Here's why.1)
Your wingmen are absolute garbage. 
They don't seem to do anything other than warn you of incoming missiles, as if the red warning lights, picture in picture, and warning buzzers weren't enough.
2) You will spent 90% of your time dodging missiles and 10% of your time TRYING to have a decent dog fight. The MIG-29's are great they are extremely tough to get a lock on but as soon as you get any where close to missile lock you have to break off in order to escape incoming missiles.3) The missions are scripted. You can't pick which targets to engage like you could in the first HAWX. If you are trying to protect a ground base from tanks you'll be forced to break off and take out incoming fighters. When you attempt to kill the fighters the ground forces practically destroy the base. Once you get rid of the fighters and try to take on the ground forces again you are taken to a cut-scene or the mission ends. That's what really gets me, the enemy doesn't retreat if you overwhelm them like the first, its just over.4) The storyline is boring. Rebel forces take over some Russian nukes, WOW that hasn't been done to death. Thankfully you can skip the load-screens after about thirty seconds.5) This isn't that big of a deal but the game is pretty sexist. 
There is a female pilot who talks quite a bit throughout the campaign and she is pretty much the only source of a "team" feel. When she talks she is almost instantly told to be quite by one of your wingmen. In the end she sacrifices herself trying to take out an enemy target and the game is like, "alright she's dead who's up next?".6) No choices of what planes or missile packages you get to fight with. You are given some really bad planes to take into combat and don't expect to play with to many US aircraft, the game is set as a world conflict so you only get to play the F-22 and Warthog in a couple missions, otherwise your Russian, French or English.The first game was great it had creative and fun missions like protecting the President of the United States and taking on wave after wave of ground targets. 
This game is intense but I would say its not very fun. There seems to have been a complete lack of imagination on this one. I have only played the multiplayer a few times and it seems to be a rank up as you go system so maybe that is worth the price of the game. All I can say is the campaign is a huge let down.ALL FEEDBACK IS APPRECIATED. THANKS AND GOOD GAMING.. Fun game but mostly arcade. This is a nice little arcade game with good graphics and some good.'''
tokens = word_tokenize(document)
topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20)
pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])

,topic #,weight,words in topic
0,7,0.09,"0.009*""world"" + 0.007*""skyrim"" + 0.005*""fallou..."
1,39,0.04,"0.063*""cable"" + 0.047*""tv"" + 0.025*""charge"" + ..."
2,41,0.02,"0.103*""arcade"" + 0.053*""collection"" + 0.052*""c..."
3,54,0.14,"0.007*""enemy"" + 0.004*""action"" + 0.003*""world""..."
4,70,0.24,"0.016*""weapon"" + 0.013*""enemy"" + 0.012*""shoot""..."
5,77,0.21,"0.021*""map"" + 0.019*""multiplayer"" + 0.017*""onl..."
6,82,0.26,"0.049*""fly"" + 0.042*""plane"" + 0.041*""mission"" ..."


In [24]:
#save model
lda_model.save("./models/100_20")

In [25]:
#output to file
output = []
for asin in data['asin']:
    #print(asin)
    output.append([asin,get_topic(asin)])
output

[['0700099867',
  ['car',
   'race',
   'racing',
   'track',
   'drive',
   'wheel',
   'speed',
   'racer',
   'online',
   'vehicle']],
 ['6050036071',
  ['puzzle',
   'ps_vita',
   'turtle_beach',
   'object',
   'word',
   'solve',
   'adventure',
   'hint',
   'scene',
   'room']],
 ['7100027950',
  ['enemy',
   'action',
   'world',
   'weapon',
   'amazing',
   'power',
   'combat',
   'short',
   'fight',
   'finish']],
 ['7293000936',
  ['cable',
   'tv',
   'charge',
   'plug',
   'xbox',
   'product',
   'charger',
   'adapter',
   'cord',
   'quality']],
 ['8176503290',
  ['fly',
   'plane',
   'mission',
   'flight',
   'aircraft',
   'sim',
   'realistic',
   'air',
   'ace_combat',
   'joystick']],
 ['907843905X',
  ['dlc',
   'option',
   'patch',
   'star',
   'bug',
   'fix',
   'choose',
   'issue',
   'side',
   'simply']],
 ['9625990674',
  ['controller',
   'mouse',
   'button',
   'charge',
   'hand',
   'battery',
   'stick',
   'product',
   'light',
   'desig

In [26]:
pd.DataFrame(output).to_csv("output.csv")